# GP regression house price prediction


Objectif: Utiliser une regresison Gaussienne pour prédire le prix d'une maison. On peut également obtenir un intervale de confiance à 2 sigma pour la prediction.


Utilisation du Dataset UCI Boston Housing

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


* CRIM: Per capita crime rate by town
* ZN: Proportion of residential land zoned for lots over 25,000 sq. ft
* INDUS: Proportion of non-retail business acres per town
* CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
* NOX: Nitric oxide concentration (parts per 10 million)
* RM: Average number of rooms per dwelling
* AGE: Proportion of owner-occupied units built prior to 1940
* DIS: Weighted distances to five Boston employment centers
* RAD: Index of accessibility to radial highways
* TAX: Full-value property tax rate per \$10,000
* PTRATIO: Pupil-teacher ratio by town
* $ B: 1000(Bk — 0.63)^2 $, where Bk is the proportion of [people of African American descent] by town
* LSTAT: Percentage of lower status of the population
* MEDV: Median value of owner-occupied homes in $1000s

## 1. Dataset

We use the UCI boston hosuing dataset

In [2]:

boston = pd.DataFrame(load_boston().data, columns=load_boston().feature_names)
boston['MEDV'] = load_boston().target
boston.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
X = pd.DataFrame(np.c_[boston['LSTAT'], boston['RM']], columns = ['LSTAT','RM'])
Y = boston['MEDV']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
X_train = torch.tensor(X_train.values).to(device)
Y_train = torch.tensor(Y_train.values).to(device)
X_test = torch.tensor(X_test.values).to(device)
Y_test = torch.tensor(Y_test.values).to(device)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

torch.Size([404, 2])
torch.Size([102, 2])
torch.Size([404])
torch.Size([102])


## 2. Model

We use a GP woth RBF kernel

In [6]:

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood().double().to(device)
model = ExactGPModel(X_train, Y_train, likelihood).double().to(device)

In [7]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 2000
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(X_train)

    # Calc loss and backprop gradients
    loss = -mll(output, Y_train)
    loss.backward()
    if i%10 == 0 :
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
        ))
    optimizer.step()

Iter 1/2000 - Loss: 40.323   lengthscale: 0.693   noise: 0.693
Iter 11/2000 - Loss: 15.573   lengthscale: 1.245   noise: 1.278
Iter 21/2000 - Loss: 9.980   lengthscale: 1.700   noise: 1.888
Iter 31/2000 - Loss: 7.993   lengthscale: 1.991   noise: 2.403
Iter 41/2000 - Loss: 7.039   lengthscale: 2.171   noise: 2.815
Iter 51/2000 - Loss: 6.477   lengthscale: 2.289   noise: 3.149
Iter 61/2000 - Loss: 6.099   lengthscale: 2.373   noise: 3.431
Iter 71/2000 - Loss: 5.811   lengthscale: 2.439   noise: 3.681
Iter 81/2000 - Loss: 5.587   lengthscale: 2.494   noise: 3.907
Iter 91/2000 - Loss: 5.400   lengthscale: 2.542   noise: 4.117
Iter 101/2000 - Loss: 5.241   lengthscale: 2.584   noise: 4.314
Iter 111/2000 - Loss: 5.100   lengthscale: 2.623   noise: 4.501
Iter 121/2000 - Loss: 4.981   lengthscale: 2.657   noise: 4.678
Iter 131/2000 - Loss: 4.872   lengthscale: 2.688   noise: 4.848
Iter 141/2000 - Loss: 4.778   lengthscale: 2.717   noise: 5.011
Iter 151/2000 - Loss: 4.691   lengthscale: 2.743 

## 3. Predictions

We predict the selling price as well as a 95% confidence interval

In [8]:

# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(0, 10, 51)
    Y_pred_dist = likelihood(model(X_test))

In [9]:
Y_pred = Y_pred_dist.mean.cpu().numpy()
conf_lo, conf_hi = Y_pred_dist.confidence_region()
conf_lo, conf_hi = conf_lo.cpu(), conf_hi.cpu()

X_test, Y_test = X_test.cpu(), Y_test.cpu()
X_train, Y_train = X_train.cpu(), Y_train.cpu()

In [10]:
preds = pd.DataFrame({'truth': Y_test.cpu(), 
                      'conf_lo': np.round(conf_lo, 1),
                      'pred': np.round(Y_pred, 1), 
                      'conf_hi': np.round(conf_hi, 1)})
preds['inside_interval'] = preds.apply(lambda row: 1 if ((row['truth'] >= row['conf_lo']) & (row['truth'] <= row['conf_hi'])) else 0, axis=1)
preds

,truth,conf_lo,pred,conf_hi,inside_interval
0,37.6,35.7,43.8,51.9,1
1,27.9,22.3,30.3,38.3,1
2,22.6,17.1,25.1,33.0,1
3,13.8,5.5,15.0,24.4,1
4,35.2,25.8,33.9,42.0,1
5,10.4,5.3,13.5,21.8,1
6,23.9,21.8,29.8,37.7,1
7,29.0,19.1,27.1,35.1,1
8,22.8,16.7,24.7,32.7,1
9,23.2,11.6,19.6,27.6,1


In [11]:
rmse = (np.sqrt(mean_squared_error(Y_test, Y_pred)))
r2 = r2_score(Y_test, Y_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print('{} / {} predictions are inside the interval'.format(sum(preds['inside_interval']), len(preds)))

The model performance for testing set
--------------------------------------
RMSE is 3.9947081397879
R2 score is 0.7961820844190546
96 / 102 predictions are inside the interval


## 2. Random Forest Regression

as a comparison

In [12]:
rfr = RandomForestRegressor(max_depth=5, random_state=0,
                             n_estimators=500)
rfr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
Y_pred_rfr = rfr.predict(X_test)

In [14]:
rmse = (np.sqrt(mean_squared_error(Y_test, Y_pred_rfr)))
r2 = r2_score(Y_test, Y_pred_rfr)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
RMSE is 3.8874798355550992
R2 score is 0.8069772292270325


In [15]:
preds = pd.DataFrame({'truth': Y_test, 
                      'pred': np.round(Y_pred_rfr, 1)})
preds

,truth,pred
0,37.6,47.3
1,27.9,27.4
2,22.6,23.8
3,13.8,10.2
4,35.2,41.8
5,10.4,13.8
6,23.9,29.8
7,29.0,25.7
8,22.8,23.1
9,23.2,19.7
